In [22]:
import pandas as pd
import sqlite3
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv("imdb_top_1000 (1).csv")

In [23]:
connection = sqlite3.connect("movies.db")
minion = connection.cursor()

In [57]:
SQL_command = """
    CREATE TABLE
        movies_data_tn
        (
            title TEXT NOT NULL,
            genre TEXT,
            release_year TEXT,
            director TEXT,
            start_1 TEXT,
            start_2 TEXT,
            runtime INTEGER,
            imdb_rating REAL NOT NULL,
            num_votes INTEGER,
            gross REAL
        );
"""
minion.execute(SQL_command)

In [25]:
tables = minion.execute("SELECT name FROM sqlite_master")
tables.fetchall()

[('movies_data',)]

In [58]:
SQL_command = """
    INSERT INTO
        movies_data_tn
    VALUES
        (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

chunksize = 100
for chunk in pd.read_csv("imdb_top_1000 (1).csv", chunksize=chunksize):
    records = []
    for index, row in chunk.iterrows():
        record = (
            row['Series_Title'],
            row['Genre'],
            row['Released_Year'],
            row['Director'],
            row['Star1'],
            row['Star2'],
            int(row['Runtime'].split()[0]),
            row['IMDB_Rating'],
            row['No_of_Votes'],
            row['Gross']
        )
        records.append(record)
    minion.executemany(SQL_command, records)
    connection.commit()

In [48]:
SQL_command = """
    SELECT
        COUNT(*)
    FROM
        movies_data;
"""
minion.execute(SQL_command)
print(minion.fetchall())

[(2000,)]


In [59]:
top10 = pd.read_sql("Select * from movies_data_tn limit 10",connection)
top10

,title,genre,release_year,director,start_1,start_2,runtime,imdb_rating,num_votes,gross
0,The Shawshank Redemption,Drama,1994,Frank Darabont,Tim Robbins,Morgan Freeman,142,9.3,2343110,"28,341,469"
1,The Godfather,"Crime, Drama",1972,Francis Ford Coppola,Marlon Brando,Al Pacino,175,9.2,1620367,"134,966,411"
2,The Dark Knight,"Action, Crime, Drama",2008,Christopher Nolan,Christian Bale,Heath Ledger,152,9.0,2303232,"534,858,444"
3,The Godfather: Part II,"Crime, Drama",1974,Francis Ford Coppola,Al Pacino,Robert De Niro,202,9.0,1129952,"57,300,000"
4,12 Angry Men,"Crime, Drama",1957,Sidney Lumet,Henry Fonda,Lee J. Cobb,96,9.0,689845,"4,360,000"
5,The Lord of the Rings: The Return of the King,"Action, Adventure, Drama",2003,Peter Jackson,Elijah Wood,Viggo Mortensen,201,8.9,1642758,"377,845,905"
6,Pulp Fiction,"Crime, Drama",1994,Quentin Tarantino,John Travolta,Uma Thurman,154,8.9,1826188,"107,928,762"
7,Schindler's List,"Biography, Drama, History",1993,Steven Spielberg,Liam Neeson,Ralph Fiennes,195,8.9,1213505,"96,898,818"
8,Inception,"Action, Adventure, Sci-Fi",2010,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,148,8.8,2067042,"292,576,195"
9,Fight Club,Drama,1999,David Fincher,Brad Pitt,Edward Norton,139,8.8,1854740,"37,030,102"


#Which movie got the highest imdb rate ?


In [60]:
highest_imdb = pd.read_sql( "Select title,imdb_rating from movies_data_tn order by imdb_rating DESC limit 1",connection)
highest_imdb

,title,imdb_rating
0,The Shawshank Redemption,9.3


#Top 10 star brings the most revenue


In [93]:
star_revenue = pd.read_sql( "Select start_1,start_2, gross from movies_data_tn group by start_1, start_2 order by gross DESC limit 10",connection)
star_revenue

,start_1,start_2,gross
0,Ranbir Kapoor,Nargis Fakhri,"985,912"
1,Danny Glover,Whoopi Goldberg,"98,467,863"
2,Buster Keaton,Kathryn McGuire,"977,375"
3,Jesse Eisenberg,Andrew Garfield,"96,962,694"
4,Liam Neeson,Ralph Fiennes,"96,898,818"
5,Toshirô Mifune,Machiko Kyô,"96,568"
6,Nicole Kidman,Christopher Eccleston,"96,522,687"
7,Elliott Gould,Nina van Pallandt,"959,000"
8,Robin Williams,Robert Sean Leonard,"95,860,116"
9,Daisy Ridley,John Boyega,"936,662,225"


#Top 10 director have highest grossing movie in the top 1000

In [86]:
least = pd.read_sql("select title, director ,gross from movies_data_tn where gross <> 'None' order by gross Asc limit 10",connection)
least

,title,director,gross
0,Cowboy Bebop: Tengoku no tobira,Shin'ichirô Watanabe,"1,000,045"
1,Hævnen,Susanne Bier,"1,008,098"
2,Short Term 12,Destin Daniel Cretton,"1,010,414"
3,Casablanca,Michael Curtiz,"1,024,560"
4,The General,Clyde Bruckman,"1,033,895"
5,Kahaani,Sujoy Ghosh,"1,035,953"
6,Dead Man,Jim Jarmusch,"1,037,847"
7,Joyeux Noël,Christian Carion,"1,054,361"
8,No Man's Land,Danis Tanovic,"1,059,830"
9,Special Chabbis,Neeraj Pandey,"1,079,369"


#Which movie is the shortest ?

In [78]:
Movie_short = pd.read_sql( 'Select title, director, runtime from movies_data_tn order by runtime DESC limit 1', connection)
Movie_short

,title,director,runtime
0,Gangs of Wasseypur,Anurag Kashyap,321


#How many movies are in the top 1000 that are more than 3 hours?

In [83]:
short = pd.read_sql("select count(*) as [movies more than 3 hours] from movies_data_tn where runtime > 180 ", connection)
short

,movies more than 3 hours
0,40
